# Read [SEAS5 Seasonal Forecast](https://www.ecmwf.int/sites/default/files/medialibrary/2017-10/System5_guide.pdf) COGs

This notebook demos basic functionality to read from our team's store of SEAS5 seasonal forecasts, stored as COGs and downloaded originally from the [ECMWF MARS service](https://www.ecmwf.int/en/forecasts/access-forecasts/access-archive-datasets).

In [1]:
import xarray as xr 
import pandas as pd
import tqdm
import os
from dotenv import load_dotenv
from azure.storage.blob import ContainerClient
import rioxarray as rxr
import io
import zipfile
import matplotlib.pyplot as plt
from datetime import datetime

load_dotenv()

DEV_BLOB_SAS = os.getenv("DSCI_AZ_SAS_DEV")
DEV_BLOB_NAME = "imb0chd0dev"
DEV_BLOB_URL = f"https://{DEV_BLOB_NAME}.blob.core.windows.net/"
GLOBAL_CONTAINER_NAME = "global"
DEV_BLOB_GLB_URL = (DEV_BLOB_URL + GLOBAL_CONTAINER_NAME + "?" + DEV_BLOB_SAS)

dev_glb_container_client = ContainerClient.from_container_url(DEV_BLOB_GLB_URL)

## 1. Load in all COGs from 2000 and join into a single `DataSet` object

Start by getting all the blob names from 2000

In [2]:
YEAR = 2000

blob_names = existing_files = [
    x.name
    for x in dev_glb_container_client.list_blobs(
        name_starts_with="mars/processed/"
    )
    if str(YEAR) in x.name
]

# For a single year's worth of data there should be 12 months * 7 leadtimes' worth of data
assert(len(blob_names) == (12*7))

Now we can loop through all of them and concatenate to create a single `DataSet` in `xarray`. 

TODO: 
- Configure `leadtime` as a dimension instead of a coordinate
- Upstream in the data processing, will likely want to include some metadata about the `values`
 

In [20]:
das = []
for blob_name in tqdm.tqdm(blob_names):
    cog_url = (
        f"https://{DEV_BLOB_NAME}.blob.core.windows.net/global/"
        f"{blob_name}?{DEV_BLOB_SAS}"
    )

    # TODO: Probably need to play with these chunk sizes 
    da_in = rxr.open_rasterio(
        cog_url, masked=True, chunks={"band": 1, "x": 225, "y": 900}
    )
    date_in = pd.to_datetime(blob_name.split(".")[0][-14:-4])
    leadtime = int(blob_name.split(".")[0][-1:])
    da_in["date"] = date_in
    da_in["leadtime"] = leadtime

    # Persisting to reduce the number of downstream Dask layers
    da_in = da_in.persist()
    das.append(da_in)

ds = xr.concat(das, dim="date", join='override', combine_attrs='drop')

100%|██████████| 84/84 [01:11<00:00,  1.17it/s]


In [21]:
ds

<xarray.DataArray (date: 84, band: 1, y: 451, x: 900)> Size: 136MB
dask.array<concatenate, shape=(84, 1, 451, 900), dtype=float32, chunksize=(1, 1, 451, 225), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 7kB -180.0 -179.6 -179.2 ... 178.8 179.2 179.6
  * y            (y) float64 4kB 90.0 89.6 89.2 88.8 ... -88.8 -89.2 -89.6 -90.0
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 672B 2000-01-01 2000-01-01 ... 2000-12-01
    leadtime     (date) int64 672B 0 1 2 3 4 5 6 0 1 2 3 ... 4 5 6 0 1 2 3 4 5 6

## 2. Plot some sample data

In [19]:
# TODO